<a href="https://colab.research.google.com/github/learn-programmers/programmers_kdt_II/blob/main/(KDT)_%EB%8D%94%EB%AF%B8_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%82%AC%EC%9A%A9%EC%9E%90_%EA%B8%B0%EB%B0%98_%EC%B6%94%EC%B2%9C_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
dummy_rating = pd.read_csv("https://grepp-reco-test.s3.ap-northeast-2.amazonaws.com/dummy_rating.csv", index_col=0)

In [3]:
dummy_rating.shape

(5, 6)

In [4]:
dummy_rating.head()

,scifi1,scifi2,scifi3,comedy1,comedy2,comedy3
user1,4.0,5.0,3.0,NaN,2.0,1.0
user2,5.0,3.0,3.0,2.0,2.0,NaN
user3,1.0,NaN,NaN,4.0,5.0,4.0
user4,NaN,2.0,1.0,4.0,NaN,3.0
user5,1.0,NaN,2.0,3.0,3.0,4.0


In [5]:
dummy_rating.fillna(0, inplace=True)
dummy_rating

,scifi1,scifi2,scifi3,comedy1,comedy2,comedy3
user1,4.0,5.0,3.0,0.0,2.0,1.0
user2,5.0,3.0,3.0,2.0,2.0,0.0
user3,1.0,0.0,0.0,4.0,5.0,4.0
user4,0.0,2.0,1.0,4.0,0.0,3.0
user5,1.0,0.0,2.0,3.0,3.0,4.0


In [6]:
# 평점 정보를 보정. 이후에 코사인 유사도를 사용하면 이는 피어슨 유사도에 해당
def standardize(row):
    new_row = (row - row.mean())/(row.max()-row.min())
    return new_row

dummy_rating_std = dummy_rating.apply(standardize)
dummy_rating_std.head()

,scifi1,scifi2,scifi3,comedy1,comedy2,comedy3
user1,0.36,0.6,0.400000,-0.65,-0.08,-0.35
user2,0.56,0.2,0.400000,-0.15,-0.08,-0.60
user3,-0.24,-0.4,-0.600000,0.35,0.52,0.40
user4,-0.44,0.0,-0.266667,0.35,-0.48,0.15
user5,-0.24,-0.4,0.066667,0.10,0.12,0.40


In [7]:
# 정규화 없이 아이템간의 유사도 측정 행렬 만들기
corrMatrix_wo_std = pd.DataFrame(cosine_similarity(dummy_rating), index=dummy_rating.index, columns=dummy_rating.index)
corrMatrix_wo_std

,user1,user2,user3,user4,user5
user1,1.000000,0.906306,0.318696,0.393893,0.431834
user2,0.906306,1.000000,0.422891,0.434613,0.515716
user3,0.318696,0.422891,1.000000,0.671249,0.925138
user4,0.393893,0.434613,0.671249,1.000000,0.760117
user5,0.431834,0.515716,0.925138,0.760117,1.000000


In [8]:
# 정규화 기반 아이템간의 유사도 측정 행렬 만들기
corrMatrix = pd.DataFrame(cosine_similarity(dummy_rating_std), index=dummy_rating.index, columns=dummy_rating.index)
corrMatrix

,user1,user2,user3,user4,user5
user1,1.000000,0.764091,-0.837183,-0.577861,-0.736161
user2,0.764091,1.000000,-0.780626,-0.601483,-0.746852
user3,-0.837183,-0.780626,1.000000,0.233344,0.641732
user4,-0.577861,-0.601483,0.233344,1.000000,0.245501
user5,-0.736161,-0.746852,0.641732,0.245501,1.000000


In [9]:
def get_similar(userId):
    similar_score = corrMatrix[userId]
    # 앞서 보정된 값을 가지고 평점의 내림차순으로 정렬
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score

In [10]:
scifi_lover = "user1"

similar_users = get_similar(scifi_lover)
similar_users.head(10)

user1    1.000000
user2    0.764091
user4   -0.577861
user5   -0.736161
user3   -0.837183
Name: user1, dtype: float64